## Import libraries

In [93]:
# Import standard libraries
import os, datetime
from pathlib import Path
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle as pl
import igraph
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import bokeh
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', 'matplotlib')
%matplotlib inline

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
output_notebook()
import seaborn as sb
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz, load_npz
# import cairo
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import zscore
from scipy.sparse.linalg import norm
import networkx as nx
from pyvis.network import Network

## Load environment variables from .env file and find project root
import sys
from dotenv import load_dotenv, find_dotenv
load_dotenv()
PROJECT_ROOT = Path(find_dotenv()).parent
# data_path = Path(PROJECT_ROOT, 'results', 'eyemap')
store_path = Path(PROJECT_ROOT, 'results','AOTU_Connectivity')
sys.path.append(str(PROJECT_ROOT.joinpath('src')))


from utils import olc_client
c = olc_client.connect(verbose=True)

from utils.celltype_conn_by_roi import CelltypeConnByRoi
from utils.celltype_conn_plotter import CelltypeConnPlotter
from utils.plotter import plot_cns, save_figure, get_skeletons, get_skeleton, get_meshes, get_mesh, show_figure
from utils.helper import slugify
from utils.neuron_bag import NeuronBag

# Import neuPrint specific libraries
from neuprint import Client, fetch_neurons, NeuronCriteria as NC, fetch_neurons, fetch_simple_connections, fetch_adjacencies, connection_table_to_matrix, merge_neuron_properties, NotNull, fetch_synapse_connections, fetch_neurons,fetch_primary_rois, fetch_all_rois, fetch_synapses, fetch_roi_hierarchy
from neuprint.utils import connection_table_to_matrix





Loading BokehJS ...

Connected to https://neuprint-cns.janelia.org[cns].
Client: neuprint-python v1.7.4
User: aishahamid201@gmail.com [readwrite]



## Get a connectivity matrix

In [94]:
# Fetch AOTU neurons and get their types
neurons_df,roi_df = fetch_neurons(NC(type='^AOTU.*'))
neurons_df   #check what you have, all AOTU neurons/unique bodyIDs

,bodyId,instance,type,pre,post,downstream,upstream,size,status,statusLabel,...,assignedOlHex2,posteriorCervicalRootPosition,receptorType,locationType,predictedNtConfidence,consensusNt,flywireType,matchingNotes,inputRois,outputRois
0,10005,AOTU019_R,AOTU019,2837,31981,23423,31981,11847745920,Traced,Roughly traced,...,None,None,None,None,0.791938,gaba,None,None,"[AOTU(R), CRE(R), CentralBrain, CentralBrain-u...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
1,10031,AOTU041_R,AOTU041,2134,20586,13446,20586,5326291658,Traced,Roughly traced,...,None,None,None,None,0.829696,gaba,None,None,"[AOTU(L), AOTU(R), CRE(L), CRE(R), CentralBrai...","[AOTU(L), AOTU(R), CentralBrain, CentralBrain-..."
2,10070,AOTU019_L,AOTU019,2836,30714,23596,30714,11809707230,Traced,Roughly traced,...,None,None,None,None,0.767173,gaba,None,None,"[AOTU(L), CRE(L), CentralBrain, CentralBrain-u...","[AOTU(L), CRE(L), CentralBrain, CentralBrain-u..."
3,10148,AOTU041_L,AOTU041,2336,19960,14187,19960,5592462575,Traced,Roughly traced,...,None,None,None,None,0.778532,gaba,None,None,"[AOTU(L), AOTU(R), BU(L), CRE(L), CentralBrain...","[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr..."
4,10212,AOTU042_L,AOTU042,3579,10679,22690,10679,4312493196,Traced,Roughly traced,...,None,None,None,None,0.824503,gaba,None,None,"[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr...","[AOTU(L), AOTU(R), CRE(L), CentralBrain, Centr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,555803,AOTU024_R,AOTU024,814,2240,6989,2240,964400165,Traced,Roughly traced,...,None,None,None,None,0.476476,acetylcholine,None,None,"[AOTU(R), ATL(L), ATL(R), CRE(R), CentralBrain...","[AOTU(R), ATL(L), ATL(R), CentralBrain, Centra..."
310,557095,AOTU008_L,AOTU008,392,2167,3198,2167,832073642,Traced,Roughly traced,...,None,None,None,None,0.542097,acetylcholine,None,ventrally projecting axonal arbors mcns,"[AOTU(L), AOTU(R), CentralBrain, CentralBrain-...","[AOTU(L), CentralBrain, CentralBrain-unspecifi..."
311,565505,AOTU008_R,AOTU008,315,2727,2802,2727,502528277,Traced,Roughly traced,...,None,None,None,None,0.613766,acetylcholine,None,ventrally projecting axonal arbors mcns,"[AOTU(R), CentralBrain, CentralBrain-unspecifi...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."
312,902249,AOTU038_R,AOTU038,111,459,704,459,278656951,Traced,Roughly traced,...,None,None,None,None,0.416629,dopamine,CB3984,None,"[AOTU(R), AVLP(R), CentralBrain, CentralBrain-...","[AOTU(R), CentralBrain, CentralBrain-unspecifi..."


In [95]:
# AOTU cell types now, unique types
AOTU_celltypes = neurons_df['type'].unique()
AOTU_celltypes

array(['AOTU019', 'AOTU041', 'AOTU042', 'AOTU023', 'AOTU012', 'AOTU035',
       'AOTU005', 'AOTU100m', 'AOTU016_c', 'AOTU061', 'AOTU052',
       'AOTU103m', 'AOTU063_b', 'AOTU064', 'AOTU033', 'AOTU101m',
       'AOTU063_a', 'AOTU049', 'AOTU027', 'AOTU014', 'AOTU015', 'AOTU046',
       'AOTU050', 'AOTU024', 'AOTU059', 'AOTU009', 'AOTU008', 'AOTU045',
       'AOTU065', 'AOTU007_c', 'AOTU002_c', 'AOTU029', 'AOTU016_a',
       'AOTU002_a', 'AOTU026', 'AOTU020', 'AOTU017', 'AOTU028',
       'AOTU016_b', 'AOTU036', 'AOTU006', 'AOTU043', 'AOTU048', 'AOTU013',
       'AOTU051', 'AOTU034', 'AOTU011', 'AOTU002_b', 'AOTU022', 'AOTU062',
       'AOTU032', 'AOTU001', 'AOTU007_b', 'AOTU047', 'AOTU003',
       'AOTU007_a', 'AOTU054', 'AOTU030', 'AOTU056', 'AOTU055', 'AOTU060',
       'AOTU102m', 'AOTU038', 'AOTU058', 'AOTU053', 'AOTU037', 'AOTU018',
       'AOTU004', 'AOTU039', 'AOTU040', 'AOTU021', 'AOTU025'],
      dtype=object)

In [96]:
#  Define pre- and post-synaptic cell types
celltypes_pre = NC(type = (None)); # shows all inputs to AOTU types
celltypes_post = AOTU_celltypes

In [97]:
#  Set connection weight threshold
threshold = 3
# Fetch synaptic connectivity between these types
neu_df, conn_df = fetch_adjacencies(celltypes_pre, celltypes_post, min_roi_weight=threshold)
# Merge with neuron properties to get types on both sides
conn_AOTU_df = merge_neuron_properties(neu_df, conn_df)


# Group by type combinations and sum the weights
conn_AOTU_df_type = conn_AOTU_df.groupby(['type_pre','type_post'])['weight'].sum().reset_index()
conn_AOTU_df_type


  0%|          | 0/2 [00:00<?, ?it/s]

,type_pre,type_post,weight
0,"(JO-C,JO-D,JO-E)",AOTU034,6
1,(JO-mz),AOTU034,8
2,5-HTPMPD01,AOTU012,8
3,5-HTPMPV01,AOTU001,5
4,5-HTPMPV01,AOTU002_a,44
...,...,...,...
8393,vCal3,AOTU051,71
8394,vCal3,AOTU053,82
8395,vCal3,AOTU065,6
8396,vDeltaA_a,AOTU046,16


In [100]:
# sort from smallest to largest
conn_AOTU_df_type_sort = conn_AOTU_df_type.sort_values(by='weight', ascending=True)
#filter for only strong connections
conn_AOTU_df_type_top = conn_AOTU_df_type_sort[conn_AOTU_df_type_sort['weight']>300]

In [103]:
# Create matrix form of the connection weights
matrix_df = conn_AOTU_df_type_top.pivot_table(index='type_pre', columns='type_post', values='weight', aggfunc='sum', fill_value=0)
# Normalize each column (post-synaptic type) by its total input
# fraction or percent of input it receives from a give type_pre among all pre types
norm_matrix_df = matrix_df.div(matrix_df.sum(axis=0), axis=1) 


In [109]:
# Hoverplot

# Prepare data, normalized
z = norm_matrix_df.values
x_labels = norm_matrix_df.columns.tolist()
y_labels = norm_matrix_df.index.tolist()

# Create hover text
hover_text = [
    [f"type_pre: {y}<br>type_post: {x}<br>weight: {z[i][j]:.3f}" 
     for j, x in enumerate(x_labels)] 
    for i, y in enumerate(y_labels)
]

# Plotly heatmap
fig = go.Figure(data=go.Heatmap(
    z=z,
    x=x_labels,
    y=y_labels,
    text=hover_text,
    hoverinfo="text",
    colorscale='Blues'
))

fig.update_layout(
    title="Normalized Connection Weights: type_pre → type_post",
    xaxis_title="type_post",
    yaxis_title="type_pre",
    width=1000,
    height=3000
)

fig.show()


# Raw weights

z = matrix_df.values
x_labels = matrix_df.columns.tolist()
y_labels = matrix_df.index.tolist()

# Create hover text
hover_text = [
    [f"type_pre: {y}<br>type_post: {x}<br>weight: {z[i][j]:.3f}" 
     for j, x in enumerate(x_labels)] 
    for i, y in enumerate(y_labels)
]

# Plotly heatmap
fig = go.Figure(data=go.Heatmap(
    z=z,
    x=x_labels,
    y=y_labels,
    text=hover_text,
    hoverinfo="text",
    colorscale='Blues'
))

fig.update_layout(
    title="Raw Connection Weights: type_pre → type_post",
    xaxis_title="type_post",
    yaxis_title="type_pre",
    width=1000,
    height=3000
)

fig.show()


print(f"Normalized Matrix → Rows (type_pre): {norm_matrix_df.shape[0]}, Columns (type_post): {norm_matrix_df.shape[1]}")
print(f"Raw Matrix → Rows (type_pre): {matrix_df.shape[0]}, Columns (type_post): {matrix_df.shape[1]}")



Normalized Matrix → Rows (type_pre): 137, Columns (type_post): 61
Raw Matrix → Rows (type_pre): 137, Columns (type_post): 61


In [ ]:
import plotly.graph_objects as go

# Step 1: Compute top 100 type_pre based on total outgoing weight
top_100_pre = conn_AOTU_df_type_top.groupby('type_pre')['weight'].sum().nlargest(100).index

# Step 2: Filter the matrix (keep original order if needed)
filtered_matrix_df = matrix_df.loc[top_100_pre]
filtered_norm_df = norm_matrix_df.loc[top_100_pre]

# --- Function to generate hover heatmap ---
def plot_hover_heatmap(df, title):
    z = df.values
    x_labels = df.columns.tolist()
    y_labels = df.index.tolist()

    hover_text = [
        [f"type_pre: {y}<br>type_post: {x}<br>weight: {z[i][j]:.3f}" 
         for j, x in enumerate(x_labels)] 
        for i, y in enumerate(y_labels)
    ]

    fig = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        text=hover_text,
        hoverinfo="text",
        colorscale='Blues'
    ))

    fig.update_layout(
        title=title,
        xaxis_title="type_post",
        yaxis_title="type_pre",
        width=1000,
        height=3000  # Adjust height for 100 rows
    )

    fig.show()

# --- Plot Normalized ---
plot_hover_heatmap(filtered_norm_df, "Top 100 type_pre (Normalized Connection Weights)")

# --- Plot Raw Weights ---
plot_hover_heatmap(filtered_matrix_df, "Top 100 type_pre (Raw Connection Weights)")


In [ ]:
# Top n number, normalize only within top n
# Here n=200
# Step 1: Compute the total weight per 'type_pre'
top_type_pre = conn_AOTU_df_type_top.groupby('type_pre')['weight'].sum().nlargest(200).index

# Step 2: Pivot the matrix for all weights
matrix_df = conn_AOTU_df_type_top.pivot_table(
    index='type_pre',
    columns='type_post',
    values='weight',
    aggfunc='sum',
    fill_value=0
)

# Step 3: Filter matrix to top 200 'type_pre'
matrix_df = matrix_df.loc[matrix_df.index.intersection(top_type_pre)]

# Step 4: Normalize the matrix (e.g., row-wise L1 normalization)
norm_matrix_df = matrix_df.div(matrix_df.sum(axis=1), axis=0).fillna(0)

# --- Plot Normalized Heatmap ---
plt.figure(figsize=(10, 10))
ax = sb.heatmap(norm_matrix_df, cmap='Blues', annot=False, linewidths=0.5)
ax.set_title('Normalized Connection Weights: type_pre → type_post')
ax.set_xlabel('type_post')
ax.set_ylabel('type_pre')
ax.set_yticks(range(len(norm_matrix_df.index)))
ax.set_yticklabels(norm_matrix_df.index, rotation=0, fontsize=7)
ax.set_xticks(range(len(norm_matrix_df.columns)))
ax.set_xticklabels(norm_matrix_df.columns, rotation=90, fontsize=7)
plt.tight_layout()
plt.show()

# --- Plot Raw Weight Heatmap ---
plt.figure(figsize=(10, 10))
ax = sb.heatmap(matrix_df, cmap='Blues', annot=False, linewidths=0.5)
ax.set_title('Connection Weights: type_pre → type_post')
ax.set_xlabel('type_post')
ax.set_ylabel('type_pre')
ax.set_yticks(range(len(matrix_df.index)))
ax.set_yticklabels(matrix_df.index, rotation=0, fontsize=7)
ax.set_xticks(range(len(matrix_df.columns)))
ax.set_xticklabels(matrix_df.columns, rotation=90, fontsize=7)
plt.tight_layout()
plt.show()